# Model Prototyping

Building the basis for our model experimentation

In [1]:
import numpy as np
import pandas as pd
import torch
import os
import json

from torch.utils import data
from torch.nn import Conv2d, AvgPool2d, ReLU, Dropout, Flatten, Linear, Sequential, Module
from torch.optim import Adam
from time import time

from tqdm import tqdm

device = torch.device("cuda:0")
torch.set_default_dtype(torch.float64)

MODELS_DIR  = '/home/cxw/sonos_rirs/models/'

In [2]:
model_dict = {}
model_dict['name'] = "testrun2_regularization"
model_dict['notes'] = "same as test run but with regularization"
model_dict['data_path'] = '/home/cxw/sonos_rirs/features/080122_5k_phase/feature_df_bp_16k.csv'
model_dict['model_path'] = os.path.join(MODELS_DIR, model_dict['name'])

In [3]:
try:
    # 尝试导入IPython
    from IPython import get_ipython
    # 检查是否在IPython环境下
    if get_ipython() is not None:
        # 加载autoreload扩展
        %load_ext autoreload
        # 设置autoreload为2
        %autoreload 2
except ImportError:
    # 如果IPython没有被安装，则不作任何操作
    pass

In [4]:
# %autoreload 2
# # import volume_estimation.modeling as model_funcs
# model_funcs.train_model(model_funcs.Baseline_Model, model_dict,\
#                         overwrite=True, epochs=10,log=False) #######################################################

In [5]:
# feat_df = pd.read_csv(model_dict['data_path'])
# model_path = os.path.join(MODELS_DIR, model_dict['name'])

# dataset = []

    
# def create_dataloader(feature_df, batch_size=1, log=True):
#     dataset = []
#     for row in tqdm(feature_df.iterrows()):
#         feat_file = row[1]['file_feature']
#         loaded = np.load(feat_file)

#         feature = loaded['feat']
#         feature = feature.reshape((1, feature.shape[0], feature.shape[1]))
#         feature = np.real(feature)

#         vol = loaded['vol']
#         if log:
#             vol = np.log10(vol)
#         dataset.append((feature, vol))
    
#     dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
#     return dataloader

# dataloader = create_dataloader(feat_df, log=False)

feat_df = pd.read_csv(model_dict['data_path'])
model_path = os.path.join(MODELS_DIR, model_dict['name'])

dataset = []

import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
import os

# 假设 csv_file_path 是你的 .csv 文件路径
csv_file_path = '/home/cxw/sonos_rirs/features/080122_5k_phase/feature_df_bp_16k.csv'

# 读取 .csv 文件
csv_df = pd.read_csv(csv_file_path)

def create_dataloader(feature_df, batch_size=1, log=False):
    dataset = []
    for row in tqdm(feature_df.iterrows()):
        feat_file = row[1]['file_feature']
        loaded = np.load(feat_file)
        
        # 获取 log_rt60
        vol = loaded['log_rt60']
        
        # 在 csv_df 中查找对应的 log_bp 值
        csv_row = csv_df[csv_df['file_feature'] == feat_file]
        if not csv_row.empty:
            log_bp_value = csv_row['log_bp'].values[0]
            vol = log_bp_value  # 用 log_bp 替换 log_rt60
        
        feature = loaded['feat']
        feature = feature.reshape((1, feature.shape[0], feature.shape[1]))
        feature = np.real(feature)
        
        if log:
            vol = np.log10(vol)
        
        dataset.append((feature, vol))
    
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
    return dataloader

# 假设 feat_df 是你的特征数据框
dataloader = create_dataloader(feat_df, log=False)

32000it [00:48, 658.88it/s]


In [6]:
savename = './testmodeldict.json'
with open(savename, 'w') as f:
    json.dump(model_dict, f)
    
with open(savename) as f:
    load_dict = json.load(f)
    
for key in load_dict.keys():
    print(key, load_dict[key])

name testrun2_regularization
notes same as test run but with regularization
data_path /home/cxw/sonos_rirs/features/080122_5k_phase/feature_df_bp_16k.csv
model_path /home/cxw/sonos_rirs/models/testrun2_regularization


In [7]:
train_df = feat_df[feat_df['split'].isin(['train','val','test'])]
val_df = feat_df[feat_df['split']=='test']
test_df = feat_df[feat_df['split']=='test']

print("Creating training dataloader")
train_dataloader = create_dataloader(train_df, batch_size=64)

print("Creating validation dataloader")
val_dataloader = create_dataloader(val_df)

print("Creating test dataloader")
test_dataloader = create_dataloader(test_df)

Creating training dataloader


32000it [00:47, 666.81it/s]


Creating validation dataloader


6380it [00:09, 662.03it/s]


Creating test dataloader


6380it [00:09, 667.35it/s]


In [8]:
features, labels = next(iter(train_dataloader))
print(f"Feature batch shape: {features.size()}")
print(f"Labels batch shape: {labels.size()}")

Feature batch shape: torch.Size([64, 1, 30, 1997])
Labels batch shape: torch.Size([64])


In [9]:
class Baseline_Model(Module):
    def __init__(self, input_shape):
        #accepts a tuple with the height/width of the feature
        #matrix to set the FC layer dimensions
        super(Baseline_Model, self).__init__()
        
        #block1
        Conv1 = Conv2d(1, 30, kernel_size=(1,10), stride=(1,1))
        Avgpool1 = AvgPool2d((1,2), stride=(1,2))

        #block2
        Conv2 = Conv2d(30, 20, kernel_size=(1,10), stride=(1,1))
        Avgpool2 = AvgPool2d((1,2), stride=(1,2))

        #block3
        Conv3 = Conv2d(20, 10, kernel_size=(1,10), stride=(1,1))
        Avgpool3 = AvgPool2d((1,2), stride=(1,2))

        #block4
        Conv4 = Conv2d(10, 10, kernel_size=(1,10), stride=(1,1))
        Avgpool4 = AvgPool2d((1,2), stride=(1,2))

        #block5
        Conv5 = Conv2d(10, 5, kernel_size=(3,9), stride=(1,1))
        Avgpool5 = AvgPool2d((1,2), stride=(1,2))

        #block6
        Conv6 = Conv2d(5, 5, kernel_size=(3,9), stride=(1,1))
        Avgpool6 = AvgPool2d((2,2), stride=(2,2))

        #dropout
        dropout_layer = Dropout(p=0.5)
        height5 = input_shape[0] - 2
        height6 = (height5 - 2) // 2

        time1 = (input_shape[1] - 9) // 2
        time2 = (time1 - 9) // 2
        time3 = (time2 - 9) // 2
        time4 = (time3 - 9) // 2
        time5 = (time4 - 7) // 2
        time6 = (time5 - 7) // 2

        flat_dims = 5 * height6 * time6
        fc_layer = Linear(flat_dims, 1)
        
        self.net = Sequential(
                    Conv1, ReLU(), Avgpool1,
                    Conv2, ReLU(), Avgpool2,
                    Conv3, ReLU(), Avgpool3,
                    Conv4, ReLU(), Avgpool4,
                    Conv5, ReLU(), Avgpool5,
                    Conv6, ReLU(), Avgpool6,
                    dropout_layer, Flatten(),
                    fc_layer, Flatten()
                )
    def forward(self, x):
        for layer in self.net:
            x = layer(x)
        return x

In [10]:
input_height = features.size()[2]
input_width = features.size()[3]

model = Baseline_Model((input_height, input_width)).to(device)

In [11]:
def MSE(output, target):
    loss = torch.mean((output - target)**2)
    return loss

def Bias(output, target):
    loss = torch.mean(torch.abs(10**output - 10**target))
    return loss

def CovStep(output, target, output_mean, target_mean):
    loss = torch.mean(((output - output_mean) * (target - target_mean)))
    return loss

def MeanAbsLogStep(output, target, log=True):
    #convert out of log
    if log:
        vol_pred = 10**output
        vol_target = 10**target
    else:
        vol_pred = output
        vol_target = target
    loss = torch.mean(torch.abs(torch.log(vol_pred/vol_target)))
    return loss

def compute_eval_metrics(dataloader, model, log=True):
    target_sum = 0
    pred_sum = 0
    n_steps = 0
    
    for (x,y) in dataloader:        
        (x, y) = (x.to(device), y.to(device))
        pred = model(x)
        target_sum += y.sum()
        pred_sum += pred.sum()
        n_steps += 1
    
    target_mean = target_sum/n_steps
    pred_mean = pred_sum/n_steps
    
    mse = 0
    mean_error = 0
    cov = 0
    abs_log_ratio = 0
    
    var_pred = 0 #technically var * N but gets cancelled out in Pearson calculation
    var_target = 0 
    
    for (x,y) in dataloader:        
        (x, y) = (x.to(device), y.to(device))
        pred = model(x)
        mse += MSE(pred, y)
        mean_error += Bias(pred, y)
        cov += CovStep(pred, y, pred_mean, target_mean)
        abs_log_ratio += MeanAbsLogStep(pred, y, log=log)
        
        var_pred += MSE(pred, pred_mean)
        var_target += MSE(y, target_mean)
        
        pears = CovStep(pred, y, pred_mean, target_mean)/(torch.sqrt(MSE(pred, pred_mean))*torch.sqrt(MSE(y, target_mean)))
    
    out_dict = {}
    out_dict['mse'] = (mse / n_steps).item()
    out_dict['bias'] = (mean_error / n_steps).item()
    out_dict['pearson_cor'] = (cov/(torch.sqrt(var_pred) * torch.sqrt(var_target))).item()
    out_dict['mean_mult'] = (torch.exp(abs_log_ratio/n_steps)).item()
    
    return out_dict
    
# eval_dict = compute_eval_metrics(val_dataloader, model)
# print(eval_dict)

In [12]:
opt = Adam(model.parameters(),lr=0.0005, weight_decay=1e-2)

hist = {
	"train_loss": [],
	"val_loss": [],
    "val_bias": [],
    "val_pearson_cor": [],
    "val_mean_mult": []
}

for ep in range(800):     #########################################################################                   
    t_start = time()
    model.train()
    
    train_loss = 0
    val_loss = 0
    train_steps = 0
    val_steps = 0
    
    for (x, y) in train_dataloader:
        (x, y) = (x.to(device), y.to(device))
        pred = model(x)
        loss = MSE(pred, y.reshape((y.shape[0], 1)))
        
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        train_loss += loss
        train_steps += 1
    
    with torch.no_grad():
        model.eval()
        
        val_metrics = compute_eval_metrics(val_dataloader, model)
    
    
    hist['train_loss'].append(train_loss/train_steps)
    hist['val_loss'].append(val_metrics['mse'])
    hist['val_bias'].append(val_metrics['bias'])
    hist['val_pearson_cor'].append(val_metrics['pearson_cor'])
    hist['val_mean_mult'].append(val_metrics['mean_mult'])
    
    t_end = time()
    
    t_elapsed = t_end - t_start
    print("Epoch: {}\tDuration: {:.2f}s\tTrain loss: {:.4f}\tVal loss: {:.4f}\tVal bias:{:.4f}\tVal Pearson correlation: {:.4e}\tVal MeanMult: {:.4f}"\
          .format(ep, t_elapsed, train_loss/train_steps, val_metrics['mse'],\
                  val_metrics['bias'], val_metrics['pearson_cor'],val_metrics['mean_mult']))
    
    
    

Epoch: 0	Duration: 124.51s	Train loss: 0.5341	Val loss: 0.2574	Val bias:818.0150	Val Pearson correlation: 1.2552e-01	Val MeanMult: 2.6007
Epoch: 1	Duration: 124.05s	Train loss: 0.2701	Val loss: 0.2597	Val bias:816.0277	Val Pearson correlation: 1.3262e-01	Val MeanMult: 2.5953
Epoch: 2	Duration: 123.76s	Train loss: 0.2685	Val loss: 0.2544	Val bias:816.1536	Val Pearson correlation: 1.4095e-01	Val MeanMult: 2.5942
Epoch: 3	Duration: 123.96s	Train loss: 0.2672	Val loss: 0.2507	Val bias:818.4637	Val Pearson correlation: 1.4970e-01	Val MeanMult: 2.6003
Epoch: 4	Duration: 123.81s	Train loss: 0.2637	Val loss: 0.2496	Val bias:815.1892	Val Pearson correlation: 1.7027e-01	Val MeanMult: 2.5886
Epoch: 5	Duration: 124.09s	Train loss: 0.2611	Val loss: 0.2475	Val bias:809.3001	Val Pearson correlation: 1.9986e-01	Val MeanMult: 2.5667
Epoch: 6	Duration: 123.80s	Train loss: 0.2568	Val loss: 0.2430	Val bias:809.1863	Val Pearson correlation: 2.3193e-01	Val MeanMult: 2.5594
Epoch: 7	Duration: 123.83s	Train l

KeyboardInterrupt: 

torch.cuda.empty_cache()

In [ ]:
import csv

# 创建一个空列表来存储pred和y的值
data_to_save = []

test1_df = test_df.sample(5)

mae = 0.0 
total_samples = 0

test_dataloader1 = create_dataloader(test1_df) 
test_random = compute_eval_metrics(test_dataloader1,model) 
print(test_random)

for(x,y) in test_dataloader: 
    (x,y) = (x.to(device),y.to(device)) 
    pred = model(x) 
    for i in range(len(pred)):
        data_to_save.append([pred[i].item(), y[i].item()])

# 指定要保存的CSV文件名
csv_filename = 'predictions.csv'

# 打开CSV文件并将数据写入
with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # 写入列名（如果需要）
    csv_writer.writerow(['Prediction', 'Actual'])
    
    # 写入数据
    csv_writer.writerows(data_to_save)

print(f'Data saved to {csv_filename}')
#     print(pred,'///',y)

#     # 计算绝对误差
#     absolute_error = torch.abs(pred-y)

#     # 累加绝对误差和样本数
#     mae += absolute_error.sum().item()
    

# #     计算平均绝对误差


# mae /= 5 
# print("MAE:", mae)



In [ ]:
with torch.no_grad():
    eval_test = compute_eval_metrics(test_dataloader, model)
    print(eval_test)

# 